# Web-scraping: сбор данных из баз данных и интернет-источников

## Управление браузером с Selenium: поиск элементов, запросы XPATH

*Алла Тамбовцева, НИУ ВШЭ*

### Часть 1: базовый поиск элементов на странице

Импортируем необходимые модули и компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запускаем браузер Chrome:

In [2]:
br = wd.Chrome()

Продолжим работать с сайтом книжного магазина «Библио-Глобус». Отправляем запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [3]:
br.get("https://www.biblio-globus.ru/")

Найдём на странице поле для ввода ключевых слов для поиска по его ID:

In [4]:
search = br.find_element(By.ID, "SearchBooks")

Введём в это поле текст – запросим книги по Python:

In [5]:
search.send_keys("Python")

Как мы уже знаем, для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

Используя метод `.send_keys()`, «отправьте» в поле `search` нажатие клавиши *Enter*. 

In [6]:
search.send_keys(Keys.ENTER)

Отлично! Теперь нужно поработать с результатами поиска. Технически, нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью BeautifulSoup. Но давайте мыслить более глобально – результаты не помещаются на одной странице, поэтому для общего решения нам надо научиться выяснять, сколько страниц пролистывать для сбора всех товаров по ключевым словам. 

Для этого нам нужно найти кнопку со стрелкой `»»` и внимательно её изучить. Давайте попробуем найти эту стрелку разными способами, чтобы познакомиться с разными атрибутами в `By`. 

Сначала найдём её последовательно по классам и тэгам (последовательно, потому что стрелка представлена ссылкой, а ссылок на странице много, нужно сначала зафиксировать более общий раздел, где её искать) и сохраним как `arrow`.

**Подсказка.** Сначала имеет смысл найти список со ссылками на разные страницы по классу.

In [7]:
ul = br.find_element(By.CLASS_NAME, "pagination")

In [8]:
arrow = ul.find_elements(By.TAG_NAME, "a")[-1]

Посмотрим, что внутри. Можем запросить весь код HTML, который есть внутри найденного объекта (его тип *WebElement*):

In [9]:
arrow.get_attribute('innerHTML')

'<span aria-hidden="true">»»</span><span class="sr-only">Next</span>'

В нашем случае полный код не очень интересен, потому что ссылка, которая отправляет нас на последнюю страницу с результатами, хранится не внутри тэгов, а в отдельном атрибуте `href` (как и обычно). Текст `»»` нам, скорее, полезен просто для того, чтобы убедиться, что мы нашли то, что нужно :) 

Запросим аналогичным образом `href`:

In [10]:
arrow.get_attribute('href')

'https://www.biblio-globus.ru/search?query=Python&page=14&sort=0&instock=&cat=0&isdiscount='

Вот эта информация уже гораздо полезнее, внутри этой ссылки есть указание на то, какая страница является последней. То есть, нам не придётся кликать на стрелки *Далее* до тех пор, пока мы не дойдём до конца, через какой-нибудь цикл while, мы сможем написать код, который будет пролистывать 10 страниц и забирать результаты. 

И да, эта ссылка, на которую мы вышли через Selenium, подсказывает нам, что и без Selenium можно справиться. Переход на каждую страницу сопровождается изменением ссылки, а значит, можно просто формировать запрос, подставляя в шаблон ссылки ключевые слова после `query` и номер страницы после `page`. Так бывает не всегда, иногда страница обновляется динамически, при активации каких-то процессов ссылка на страницу остаётся та же, а её содержимое меняется – в код HTML «встраиваются» новые объекты. Тогда без Selenium не обойтись, потому что без имитации конкретных действий необходимые объекты просто не появятся (страница или её часть просто не обновится, так как не запустится соответствующая функция на JavaScript, которая реагирует на действия пользователя).

**Вопрос.** Подумайте, как можно организовать код без использования Selenium, чтобы запрос пользователя отправлялся на сайт магазина и чтобы ему возвращались все результаты по запросу. 

Со страницами результатов разобрались, к ним мы вернёмся, чтобы забрать исходный код и ссылки на товары, а пока давайте посмотрим на ещё один способ поиска объектов – найдём нужную стрелку проще – по тексту ссылки.

In [11]:
br.find_element(By.PARTIAL_LINK_TEXT, "»»")

<selenium.webdriver.remote.webelement.WebElement (session="139e5a85551e19e7c47887d915754f35", element="EB72928E73BABC011BB681466A3D2638_element_65")>

Использованные выше способы поиска хорошие, но не всегда надёжные и универсальные. В первом случае смущает поиск элемента внутри другого, да ещё и по индексу. Во втором случае потенциальных проблем меньше, но не всегда на нужной кнопке будет текст, там вообще может быть картинка. Поэтому давайте познакомимся с универсальным способом поиска через запросы XPATH.

### Часть 2: поиск с помощью запросов XPATH

Немного теории.

**XML** (от *eXtended MarkUp Language*) – язык разметки, только в отличие от HTML не позволяет регулировать внешний вид страницы, а просто хранит данные в виде строки с удобными тэгами.

На XML-файл можно смотреть как на хранилище, откуда по запросу динамически подгружаются данные для подстановки в HTML-файл. Смысл: когда нам нужно постоянно обновлять информацию на веб-странице (каталоги товаров в магазине, данные о погоде, курсе валют), не нужно каждый раз переписывать HTML-файл, достаточно изменить XML-файл, а из него уже информация «подтянется» на страницу с помощью запроса, написанного на JavaScript.

**XPATH** (от *XML Path Language*) – язык запросов в XML-файлу, который можно использовать и для HTML тоже.

Примеры запросов (честно взяты [отсюда](https://www.w3schools.com/xml/xpath_intro.asp), очень полезный тьюториал по XPATH, у них же есть классные материалы по XML):

* `//title[@lang]`: все элементы с тэгом `<title>`, имеющие атрибут `lang`;
* `//title[@lang='en']` : все элементы с тэгом `<title>`, имеющие атрибут `lang`, равный `'en'`;
* `//title[@*]`: все элементы с тэгом `<title>`, имеющие хоть какие-нибудь атрибуты.


Давайте воспользуемся поиском с помощью запроса XPATH и найдём стрелку-ссылку на последнюю страницу (внимание на атрибут `aria-label`).

In [12]:
# поиск по атрибуту aria-label
# проблема – элемент с Next не один

br.find_elements(By.XPATH, "//a[@aria-label='Next']")

[<selenium.webdriver.remote.webelement.WebElement (session="139e5a85551e19e7c47887d915754f35", element="EB72928E73BABC011BB681466A3D2638_element_64")>,
 <selenium.webdriver.remote.webelement.WebElement (session="139e5a85551e19e7c47887d915754f35", element="EB72928E73BABC011BB681466A3D2638_element_65")>]

In [13]:
# более надежный способ – найти через текст »»
# ищем тэг span внутри тэга a (отсюда .// – точка отвечает за вложенность),
# такой, что текст внутри равен »»

a = br.find_element(By.XPATH, "//a[.//span[text()='»»']]")

In [14]:
# вот ссылка на последнюю страницу – забираем
# атрибут href
# а в самой ссылке есть номер последней страницы
# будет понятно, сколько раз запускать цикл for для пролистывания

a.get_attribute("href")

'https://www.biblio-globus.ru/search?query=Python&page=14&sort=0&instock=&cat=0&isdiscount='

### Часть 3: собираем всё вместе и сохраняем результаты

Итак, допустим, у нас есть переменная `to_search` с ключевыми словами для поиска.  

In [15]:
to_search = "Python"

Импортируем вспомогательные модули и функции:

In [16]:
import re
from bs4 import BeautifulSoup
from time import sleep

Напишем код, который будет выполнять поиск по ключевым словам и сохранять ссылки на страницы товаров.

In [17]:
# начало кода из ячеек выше – открываем страницу
# отправляем ключевые слова и забираем номер последней 
# страницы с результатами

br.get("https://www.biblio-globus.ru/")
sleep(1)

search = br.find_element(By.ID, "SearchBooks")
search.send_keys(to_search)
search.send_keys(Keys.ENTER)

arrow = br.find_element(By.XPATH, "//a[.//span[text()='»»']]")
link = arrow.get_attribute("href")

# выпендриваемся, вместо нескольких .split() 
# пользуемся регулярными выражениями

n = re.search('(page=)(\d+)', link).group(2)

# для шаблона
# https://www.biblio-globus.ru/search?query=Python&page=9&sort=0&instock=&cat=0&isdiscount=

# в soups – объекты bs с кодом HTML для каждой страницы
# с результатами

soups = []

for i in range(1, int(n) + 1):
    req = f"https://www.biblio-globus.ru/search?query={to_search}&page={i}&sort=0&instock=&cat=0&isdiscount="
    br.get(req)
    sleep(2)
    html = br.page_source
    soup = BeautifulSoup(html)
    soups.append(soup)

Пишем функцию, которая забирает из кода HTML с одной страницей результатов поиска список ссылок:

In [18]:
def get_books(s):
    """
    Parameters:
        s – BeautifulSoup object with HTML code
        for one page with results
    Returns:
        hrefs – list of strings with links
    """
    prods = s.find_all("div", {"class" : "product"})
    hrefs = [p.find("a").get("href") for p in prods]
    return hrefs

Применяем функцию ко всем фрагментам кода в `soups`:

In [19]:
res = [get_books(s) for s in soups]

Получили список списков, где один список – список неполных ссылок на товары с каждой страницы результатов:

In [20]:
print(res[0])

['/product/10829080', '/product/10780215', '/product/10898149', '/product/10954900', '/product/10263752', '/product/10938051', '/product/10973493', '/product/10952299', '/product/10917575', '/product/10569801', '/product/10532193', '/product/10548788']


Теперь можем распаковать списки и доклеить к каждой ссылке начало `"https://www.biblio-globus.ru"`. Чтобы избежать цикла в цикле для перебора ссылок, импортируем модуль `itertools` и возьмём оттуда функцию `from_iterable()`, она умеет склеивать список вложенных списков в единый список:

In [22]:
import itertools

In [24]:
links = list(itertools.chain.from_iterable(res))
print(links[0:20])

['/product/10829080', '/product/10780215', '/product/10898149', '/product/10954900', '/product/10263752', '/product/10938051', '/product/10973493', '/product/10952299', '/product/10917575', '/product/10569801', '/product/10532193', '/product/10548788', '/product/10877910', '/product/10754289', '/product/10829190', '/product/10977442', '/product/10823441', '/product/10741680', '/product/10839595', '/product/10627655']


In [25]:
links_full = ["https://www.biblio-globus.ru" + link for link in links]
print(links_full[0:20])

['https://www.biblio-globus.ru/product/10829080', 'https://www.biblio-globus.ru/product/10780215', 'https://www.biblio-globus.ru/product/10898149', 'https://www.biblio-globus.ru/product/10954900', 'https://www.biblio-globus.ru/product/10263752', 'https://www.biblio-globus.ru/product/10938051', 'https://www.biblio-globus.ru/product/10973493', 'https://www.biblio-globus.ru/product/10952299', 'https://www.biblio-globus.ru/product/10917575', 'https://www.biblio-globus.ru/product/10569801', 'https://www.biblio-globus.ru/product/10532193', 'https://www.biblio-globus.ru/product/10548788', 'https://www.biblio-globus.ru/product/10877910', 'https://www.biblio-globus.ru/product/10754289', 'https://www.biblio-globus.ru/product/10829190', 'https://www.biblio-globus.ru/product/10977442', 'https://www.biblio-globus.ru/product/10823441', 'https://www.biblio-globus.ru/product/10741680', 'https://www.biblio-globus.ru/product/10839595', 'https://www.biblio-globus.ru/product/10627655']
